# BG Action Selection with FUS (Interactive)
Colab/VS Code notebook. Use the sliders to explore how FUS attenuation changes action selection.

In [ ]:
try:
    import google.colab  # type: ignore
    %pip -q install nengo matplotlib ipywidgets
except Exception:
    pass

In [ ]:
import nengo, numpy as np, matplotlib.pyplot as plt
from ipywidgets import FloatSlider, interact
np.random.seed(0)

def run_sim(A1=0.8, A2=0.6, kappa=0.6, fus_start=0.5, fus_dur=0.3, T=1.2):
    def fus_scale(t):
        return (1.0 - kappa) if (fus_start <= t <= fus_start+fus_dur) else 1.0
    model = nengo.Network(label="BG+Thal FUS")
    with model:
        cortex = nengo.Node(lambda t: [A1 * fus_scale(t), A2], label="Cortex [A1,A2]")
        bg = nengo.networks.BasalGanglia(dimensions=2)
        th = nengo.networks.Thalamus(dimensions=2)
        nengo.Connection(cortex, bg.input, synapse=0.02)
        nengo.Connection(bg.output, th.input, synapse=0.02)
        p_ctx = nengo.Probe(cortex)
        p_bg  = nengo.Probe(bg.output, synapse=0.05)
        p_th  = nengo.Probe(th.output, synapse=0.05)
    with nengo.Simulator(model, seed=1) as sim:
        sim.run(T)
    t = sim.trange()
    th_y = sim.data[p_th]
    plt.figure(figsize=(8,5))
    plt.subplot(3,1,1)
    plt.plot(t, sim.data[p_ctx][:,0], label="A1 (FUS-attenuated)")
    plt.plot(t, sim.data[p_ctx][:,1], label="A2")
    plt.axvspan(fus_start, fus_start+fus_dur, alpha=0.15, label="FUS")
    plt.legend(); plt.ylabel("Cortex")
    plt.subplot(3,1,2)
    plt.plot(t, sim.data[p_bg]); plt.ylabel("BG out")
    plt.subplot(3,1,3)
    plt.plot(t, th_y); plt.ylabel("Thalamus"); plt.xlabel("Time (s)")
    plt.tight_layout(); plt.show()

interact(run_sim,
         A1=FloatSlider(min=0.0,max=1.0,step=0.05,value=0.8),
         A2=FloatSlider(min=0.0,max=1.0,step=0.05,value=0.6),
         kappa=FloatSlider(min=0.0,max=1.0,step=0.05,value=0.6),
         fus_start=FloatSlider(min=0.0,max=1.0,step=0.05,value=0.5),
         fus_dur=FloatSlider(min=0.05,max=0.7,step=0.05,value=0.3),
         T=FloatSlider(min=0.6,max=2.0,step=0.1,value=1.2));